### Step 1: Read in relevant data sets

In [5]:
import os
from openpyxl import load_workbook
import pandas as pd

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')

xls_filename = 'batteries.xlsm'
input_ws_name = 'Basic Inputs'
header_col_num = 6 

In [55]:
wb = load_workbook(os.path.join(DATA_DIR, xls_filename), data_only=True)
input_ws = wb[input_ws_name]

input_ws_vals = [row for row in input_ws.values]
input_ws_headers = input_ws_vals[header_col_num]

date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'
total_vre_gen_colname = 'total_vre_mw'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'

input_ws_cols = (date_colname, load_colname, wind_gen_colname, solar_gen_colname)

In [61]:
orig_load_and_gen_df = pd.DataFrame(data= input_ws_vals[header_col_num + 1:], columns= input_ws_headers)

load_and_gen_df = orig_load_and_gen_df.copy()
load_and_gen_df = load_and_gen_df.iloc[:, [i for i in range(len(input_ws_cols))]]
load_and_gen_df.columns = input_ws_cols
load_and_gen_df.head()

,date,load_mw,wind_mw,solar_mw
0,2017-01-01 00:00:00.000000,35.1,1.67966,-0.007002
1,2017-01-01 01:00:00.000000,35.1,1.67966,-0.007002
2,2017-01-01 02:00:00.000000,34.6,1.23327,-0.007002
3,2017-01-01 02:59:59.999999,34.2,1.09193,-0.007002
4,2017-01-01 03:59:59.999999,33.8,1.46510,-0.007002


### Step 2: Prep for analysis
* clean vals in existing cols
* add columns derived from prev cols

In [65]:
# drop all rows that have all NaNs
load_and_gen_df = load_and_gen_df.dropna(how='all')

# all solar vals need to be > 0
load_and_gen_df[solar_gen_colname] = load_and_gen_df[solar_gen_colname]\
    .apply(lambda x: x if x > 0 else 0)

# format date to be month/day/year hour:min
# dt_format = '%m/%d/%Y %H:%M'
# load_and_gen_df[date_colname] = load_and_gen_df[date_colname].dt.strptime(dt_format)

# add wind and solar to get total VRE / hr
load_and_gen_df[total_vre_gen_colname] = load_and_gen_df[wind_gen_colname] + load_and_gen_df[solar_gen_colname]

assert len(load_and_gen_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

In [67]:
load_and_gen_df[month_colname] = load_and_gen_df[date_colname].map(lambda x: x.month)
load_and_gen_df[day_of_month_colname] = load_and_gen_df[date_colname].map(lambda x: x.day)
load_and_gen_df[day_of_yr_colname] = load_and_gen_df[date_colname].map(lambda x: x.timetuple().tm_yday)

In [68]:
load_and_gen_df

,date,load_mw,wind_mw,solar_mw,total_vre_mw,month,day_of_month,day_of_yr
0,2017-01-01 00:00:00.000000,35.1,1.67966,0.0,1.67966,1,1,1
1,2017-01-01 01:00:00.000000,35.1,1.67966,0.0,1.67966,1,1,1
2,2017-01-01 02:00:00.000000,34.6,1.23327,0.0,1.23327,1,1,1
3,2017-01-01 02:59:59.999999,34.2,1.09193,0.0,1.09193,1,1,1
4,2017-01-01 03:59:59.999999,33.8,1.46510,0.0,1.46510,1,1,1
...,...,...,...,...,...,...,...,...
8755,2017-12-31 18:59:59.998165,45.5,6.67250,0.0,6.67250,12,31,365
8756,2017-12-31 19:59:59.998165,46.5,6.76534,0.0,6.76534,12,31,365
8757,2017-12-31 20:59:59.998165,45.0,6.20556,0.0,6.20556,12,31,365
8758,2017-12-31 21:59:59.998165,41.7,8.72619,0.0,8.72619,12,31,365


,date,load_mw,wind_mw,solar_mw,month,day,day_of_month
0,2017-01-01 00:00:00.000000,35.1,1.67966,-0.007002,1.0,1.0,1.0
1,2017-01-01 01:00:00.000000,35.1,1.67966,-0.007002,1.0,1.0,1.0
2,2017-01-01 02:00:00.000000,34.6,1.23327,-0.007002,1.0,1.0,1.0
3,2017-01-01 02:59:59.999999,34.2,1.09193,-0.007002,1.0,1.0,1.0
4,2017-01-01 03:59:59.999999,33.8,1.46510,-0.007002,1.0,1.0,1.0
...,...,...,...,...,...,...,...
8756,2017-12-31 19:59:59.998165,46.5,6.76534,-0.007002,12.0,31.0,31.0
8757,2017-12-31 20:59:59.998165,45.0,6.20556,-0.007002,12.0,31.0,31.0
8758,2017-12-31 21:59:59.998165,41.7,8.72619,-0.007002,12.0,31.0,31.0
8759,2017-12-31 22:59:59.998165,39.6,7.23441,-0.007002,12.0,31.0,31.0
